# Disease diagnosis assistant

This noteboook will cover a disease diagnosis classifcation algorithm for diagnosis assistance. The algorithm is based off of a data set that is more than a decade old and is therefore more of a proof of concept. The recommendation afterwards is for hospitals to pull together and create a proper open source dataset with disease sympotms so that various aplications can be applied, one of which is a online tool to help individuals with self diagnosis. Especially during a time when testing is short, the output of number of cases with COIVD-19 as a possible diagnosis could be collected, along with the users location, to give an approximation on possible cases. This could help in mitigating disease and also help provide proper resources to those who need it.

The dataset used is from columbia http://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html and was collected based on patient discharge summaries at New York Presbyterian Hospital from 2004. The total amount of patient discharge included in the dataet collection was 33,724. The dataset includes the disease, the symptoms, and the number of disseases ocurring. The dataset contains a total of 133 diseases and their associated symptoms. The dataset version imported into this project does not include number of disease occurances as it is not used in the classification algorithm. The original dataset had codes for identification such as UMLS:C0008031 that were removed in excel using text-to-columns as part of a data cleaning step.

The diease was populated for each symptom for classification by filling in blank cells with the appropriate diseases. https://www.ablebits.com/office-addins-blog/2014/05/02/fill-blanks-excel/

In [1]:
import pandas as pd
import numpy as np

In [2]:
#reading in dataframe
data = pd.read_csv('diseases.csv') 

In [3]:
#display top rows of dataframe to confirm upload is correct
data.head()

,Case,ID,Disease,Symptom
0,1,1,hypertensive disease,pain chest
1,1,1,hypertensive disease,shortness of breath
2,1,1,hypertensive disease,dizziness
3,1,1,hypertensive disease,asthenia
4,1,1,hypertensive disease,fall


With the data inputted properly, lets examine the dataframe.

In [4]:
data_disease= data.drop(columns=['ID','Case'])
data_disease.head()

,Disease,Symptom
0,hypertensive disease,pain chest
1,hypertensive disease,shortness of breath
2,hypertensive disease,dizziness
3,hypertensive disease,asthenia
4,hypertensive disease,fall


In [5]:
data_disease.describe()

,Disease,Symptom
count,48760,48760
unique,133,396
top,upper respiratory infection,shortness of breath
freq,714,1201


From the table above, it can be determined that there are a total of 133 diseases classifed in the dataset and 396 unique symptoms. There sre 1864 entries in the symptom column with only 396 unique symptom values, meaning there are many common symoptoms between diseases. This is why a machine learning classification algorithm is needed in order to provide the most likely disease from a collection of symptoms.

Next, lets explore the different diseases to see which disease have the most associated symptoms.

In [6]:
data['Disease'].value_counts()

upper respiratory infection             714
bipolar disorder                        702
anxiety state                           648
psychotic disorder                      633
malignant neoplasms                     626
malignant neoplasm of prostate          604
suicide attempt                         586
chronic kidney failure                  572
hypothyroidism                          567
emphysema pulmonary                     566
COVID-19                                548
depression mental                       547
anemia                                  528
diverticulitis                          512
confusion                               511
pneumonia                               505
carcinoma                               497
hemiparesis                             486
personality disorder                    485
delusion                                483
Pneumocystis carinii pneumonia          479
myocardial infarction                   474
primary carcinoma of the liver c

It would be helpful to also know which are the most common symptoms among all diseases.

In [7]:
data['Symptom'].value_counts()

shortness of breath         1201
pain                        1073
fever                        912
pain abdominal               705
diarrhea                     629
vomiting                     625
asthenia                     581
unresponsiveness             581
cough                        578
nausea                       574
dyspnea                      574
chill                        546
decreased body weight        525
pain chest                   520
apyrexial                    502
agitation                    486
rale                         471
lesion                       450
mass of body structure       449
hypotension                  419
sore to touch                419
syncope                      398
thicken                      372
orthopnea                    367
hallucinations auditory      365
night sweat                  365
haemorrhage                  346
swelling                     341
tremor                       340
feeling suicidal             336
          

We can examine the top 10 most common symptoms to get a clearer picture.

In [8]:
data_count= data.Symptom.value_counts().reset_index().rename(columns={'index': 'Symptom type', 0: 'count'})
data_count.head()

,Symptom type,Symptom
0,shortness of breath,1201
1,pain,1073
2,fever,912
3,pain abdominal,705
4,diarrhea,629


In [9]:
data_count = data_count.rename(columns = {"Symptom type": "Symptom", 
                                  "Symptom":"Count"}) 
top_10 = data_count.head(10)
top_10.head()

,Symptom,Count
0,shortness of breath,1201
1,pain,1073
2,fever,912
3,pain abdominal,705
4,diarrhea,629


In [10]:
import matplotlib.pyplot as plt
figure = plt.figure()
axes = figure.add_axes([1,1,2,2])
X = top_10['Symptom']
Y= top_10['Count']
axes.bar(X,Y)
axes.set_ylabel('Number of diseases with symptom')
axes.set_xlabel('Disease symptom')
axes.set_title('Most common disease symptoms')
plt.show()

<Figure size 640x480 with 1 Axes>

In [11]:
# One-hot encode the data using pandas get_dummies
data_onehot = pd.get_dummies(data.Symptom)
# Display the first 5 rows of the last 12 columns
data_onehot.iloc[:,5:].head(5)

,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse reaction,agitation,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data_onehot.shape

(48760, 396)

In [13]:
# add neighborhood column back to dataframe
data_onehot['Case'] = data['Case'] 

# move neighborhood column to the first column
fixed_columns = [data_onehot.columns[-1]] + list(data_onehot.columns[:-1])
data_onehot = data_onehot[fixed_columns]

data_onehot.head()

,Case,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_onehot.head(15)

,Case,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data_onehot.shape

(48760, 397)

In [16]:
data_grouped = data_onehot.groupby('Case').sum().reset_index()
data_grouped.head()


,Case,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [17]:
data_grouped.shape


(3570, 397)

First thing of note here is that we can see that there are 3570 unique cases in the original dataset.

Most importantly, we now have a data set where each unique case had it's associated sympotms in the corresponding rows, marked 1 if the symptom is present and 0 if not. Next, we need to add in the disease diagnosis for each case. We'll do that by joining the dataset with a dataset that has each unique case with it's corresponding diagnoses ID. 

In [18]:
#reading in dataframe
case_id = pd.read_csv('Case ID.csv') 

In [19]:
case_id.head()

,ID,Number,Disease,Symptom
0,1,1,hypertensive disease,pain chest
1,2,2,diabetes,polyuria
2,3,3,depression mental,feeling suicidal
3,4,4,coronary arteriosclerosis,pain chest
4,5,5,pneumonia,cough


In [20]:
diagnoses = pd.merge(data_grouped, case_id, left_on="Case", right_on="Number")
diagnoses.head()

,Case,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,...,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,ID,Number,Disease,Symptom
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,hypertensive disease,pain chest
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,diabetes,polyuria
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,3,3,depression mental,feeling suicidal
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4,4,coronary arteriosclerosis,pain chest
4,5,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,5,5,pneumonia,cough


In [21]:
diagnoses= diagnoses.drop(columns=['Case','Number','Disease','Symptom'])
diagnoses.head()

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,...,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum,ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,5


In [22]:
# move neighborhood column to the first column
fixed_columns = [diagnoses.columns[-1]] + list(diagnoses.columns[:-1])
diagnoses = diagnoses[fixed_columns]

diagnoses.head()

,ID,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [23]:
diagnoses.shape

(3570, 397)

Fantastic! We now have 3570 unique cases in the form of the symptom and the ultimate outcome. The data is now ready for machine learning.

In [24]:
#convert into arrays

# Labels are the values we want to predict
labels = np.array(diagnoses['ID'])
# Remove the labels from the features
# axis 1 refers to the columns
features= diagnoses.drop('ID', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [25]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 1)

In [26]:
print('train features:', train_features.shape)
print('train labels:', train_labels.shape)
print('test features:', test_features.shape)
print('test labels', test_labels.shape)

train features: (2856, 396)
train labels: (2856,)
test features: (714, 396)
test labels (714,)


From the above we can see that the training features have 396 symptom columns, and the labels have the same number of rows but only 1 column. This is the format we want.

In [27]:
from sklearn.ensemble import RandomForestClassifier
#we will delpoy a random forest classifier which is an enhancement of the decision tree model and is a good classifier for this situation

model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
  

In [28]:
## TODO: Train the model
    
model.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
# Use the forest's predict method on the test data
predictions = model.predict(test_features)



In [30]:
from sklearn.metrics import accuracy_score

accuracy_score(test_labels, predictions)

0.9915966386554622

We see that the accuracy score is extremely high. Normally, the accuracy would be lower, but since this dataset is a mock dataset and not real cases, the test cases are most likely pretty simliar to the trained cases, and thus it will get most right. The test cases are very similar becuase the cases were differentiated by random deletion. In real instances, the cases would be more varied.

That said, the model is fully functioning and ready to take an input. This brings up an entirely new challenge. If this model is to be correctly deployed, it needs take an input from an enlgish language sentence descirbing symptoms. Obviously the model can't decipher english, so the text will need to be converted into the correct fromat. Thus, a function needs to be created that converts a plan text string into the onehot format the model knows.

In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

# the following function will convert the text string into an input for the model
def format_input(symptoms):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(symptoms, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    #now that the sentence has been converted into a list of words, it can be converted into the onehot format for input
    
    #first, we need to build a data frame from the word list with "Symptom" as the only column
    d = {'Symptom': words } 
    df = pd.DataFrame(data=d)  
    df_onehot = pd.get_dummies(df.Symptom) #getting binary values for the symptoms rather than strings for numerical formatting.
    
    #add a column with 1 in every row so it can be grouped by that value to get all the symptom 1's in one row
    df_onehot['Value'] = '1' 
    fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
    df_onehot = df_onehot[fixed_columns]
    df_grouped = df_onehot.groupby('Value').sum().reset_index()
    
    #drop value column as it is now unneeded 
    df_grouped = df_grouped.drop(columns=['Value'])
    
    #need to match the symptoms in the description with the symptoms in the master dataset
    #grab the symptom list from the original dataset
    data_to_match  = pd.get_dummies(data.Symptom)
    data_to_match = data_to_match.iloc[[0]]
    
    #joining the user symptoms list with the total symptom list
    user_input = pd.concat([data_to_match, df_grouped], join = 'outer', sort=True)
    #Setting new index so rows can be differentiated
    user_input = user_input.set_index([pd.Index([0, 1])])
    #drop original datset symptoms
    user_input_1 = user_input.drop([0], axis =0)
    #fill missing values
    user_input_1 = user_input_1.fillna(0)
    #rejoin datasets so that the only column with missing values are words that weren't symptoms
    user_input_1 = pd.concat([data_to_match, user_input_1], join = 'outer', sort=True)
    #dropping all non-symptom words
    user_input_1 = user_input_1.dropna(axis=1)
    #redropping the extra row with asmaple symptoms from the original dataset - user input is now in correct format
    user_input_1 = user_input_1.drop([0], axis =0)
    
    return user_input_1

In [32]:
# Test the format_input function on a sample diagnosis
symptoms = format_input('I am experiencing various symptoms. The main ones I can think of are fever, chill, a sneeze, snuffle, headache,sore throat, blackout, extreme exhastion, and debilitation.')
symptoms

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,...,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we can pass the input into the model to see what the resulting prediction is.

In [34]:
prediction = model.predict(symptoms)
prediction


array([120])

The model successfully predicted a disease value. This is great! However, the disease value is in numeric ID form. We want the disease name printed. We can create a function that takes the array value can returns the disease name so we can determine what the dignosis is in plain english.

In [35]:
def disease(prediction):
    #search ID dataset for the ID that matches the prediction and return the disease column 
    df= (case_id.Disease[case_id['ID'] == prediction[0]])
    #Get the unique disease value
    df=df.unique()
    df= df[0]
    return df

In [36]:
print("You may have", disease(prediction),".")

You may have COVID-19 .


A model has now successfully been created that can take in an english langauge description of symptoms and print out a predicted diagnosis.